In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import re
import os
import pdfplumber
import json


In [29]:
system_prompt = """
Given a set of technical specifications, please generate a corresponding JSON representation in this format/Structure: {"heading": "Technical specifications heading", "values": [/* List of specific values */]}. Let the 'heading' serves as a label, and 'values' represent specific details under each category. If no heading is included in the text, make your own headings that makes sense for the technical specifications and that groups several of the values. Add each attribute as a specific values in the list. Use the following example and format as a guide:

{
  "specifications": {
    "Component Specifications": [
      {"Component Name": "[Value]"},
      {"Another Component": "[Value]"}
    ],
    "Environmental Specifications": [
     { "Operating temperature": "[Min-Max] [Unit]"},
      {"Storage temperature": "[Min-Max] [Unit]"},
      {"Operating humidity": "[Max] [Unit] non-condensing"},
      {"Storage humidity": "Less than [Max] [Unit]"}
    ],
    "Power Requirements": [
      {"Voltage": "[Range] [Frequency] [Unit]"},
      {"Power consumption": "[Max] [Unit]"}
    ],
    "Interfaces": [
      {"Interface Type": "[Count]"},
      {"Another Interface": "[Count]"}
    ],
    "Data Outputs": [
      {"Message format": "[Format]"},
      {"Message type": "[Type]"}
    ]
  }
}
""" 

In [26]:
def generate_technical_specs(text):
    import os
    from openai import OpenAI
    from openai import AzureOpenAI
    import pandas as pd


    deployment_name = "gpt-4-turbo-1106-preview"  # This will correspond to the custom name you chose for your deployment when you deployed a model.


    client = AzureOpenAI(
        azure_endpoint="https://da-openai-test.openai.azure.com/",
        api_key="5a90b85ba420469c9b36438e238d70fe",
        api_version="2023-05-15",
    )

    response = client.chat.completions.create(
        model=deployment_name,
        messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": f"""Given this set of technical specifications {text}, please generate a corresponding JSON representation in this format/Structure: "heading": "Techniocal specifications heading", "values": [/* List of specific values */]."""
                }
            ],
        response_format={"type": "json_object"},
    )

    return response.choices[0].message.content.strip()



In [27]:
number = 1
products = pd.read_excel("data/all_products_dropped_discontinuid.xlsx")
print(products.iloc[number]["Data sheets"])
print(products.iloc[number]["Technical Specifications"])
print(generate_technical_specs(products.iloc[number]["Technical Specifications"]))

https://www.kongsberg.com/contentassets/d106c9aa05de423a8b02cf5fae6d6ad1/160920_transducer_12-16_60.pdf
Resonant frequency: 12 kHz
Maximum transducer depth: 20 m
Cable length: 20 m
Cable diameter: 12.5 mm
Weight: 84 / 78 kg with/without cable
Storage temperature: -20 to 70 °C
Circular beamwidth: 16 deg (narrow), 60 deg (wide)
Directivity: 130 (narrow), 10 (wide)
Impedance: Nominal 60 ohms
Transmitting response: 142 dB re 1µPa per V
Receiving sensitivity: -168.5 dB re 1V per µPa
Electroacoustic efficiency: 0.60
Maximum pulse power input: 100 W
Maximum continuous power input: 4 W
{
  "specifications": {
    "Acoustic Performance": [
      {"Resonant frequency": "12 kHz"},
      {"Circular beamwidth": "16 deg (narrow), 60 deg (wide)"}
    ],
    "Transducer Characteristics": [
      {"Maximum transducer depth": "20 m"},
      {"Directivity": "130 (narrow), 10 (wide)"},
      {"Impedance": "Nominal 60 ohms"},
      {"Transmitting response": "142 dB re 1µPa per V"},
      {"Receiving sensit

In [34]:
# products = pd.read_excel("data/all_products_dropped_discontinuid.xlsx")
# products["formated_technical_specs"]=""
# products["formated_technical_specs"] = products["formated_technical_specs"].astype(str)
# products.to_excel("data/all_products_technical_formated.xlsx")


In [47]:
import pandas as pd

# Load the products data
products = pd.read_excel("data/all_products_technical_formated.xlsx")

# Iterate over each row in the DataFrame
for index, row in products.iterrows():
        if pd.isna(row["formated_technical_specs"]):
            if not pd.isna(row["Technical Specifications"]):

                print(row["Product_Name"], "index:", index)
                print(row["Data sheets"])
                print(row["Technical Specifications"])
                json_output = (generate_technical_specs(row["Technical Specifications"]))
                json_output = json_output.replace('\n', '\n').replace('\r', '\n').replace('\t', '\n')

                try:
                    data = json.loads(json_output)
                    print(data)
                    products.loc[index, 'formated_technical_specs'] = json.dumps(data)
                except json.JSONDecodeError:
                    print(f'Error decoding JSON for index {index}: {json_output}')

                products.to_excel("data/all_products_technical_formated.xlsx")
                products.to_csv("data/all_products_technical_formated.csv")

EM 2040 PHS Portable Hydrographic System  index: 70
https://www.kongsberg.com/contentassets/e7694c05ba004432a68c918dff57c1fb/multibeam-intensity-v2022.pdf
System SL NF1 PL@NF1 NF2 PL@NF2 SBP 120/300/27/29 3° 230 0.8 m 209 138 m 187 SBP System SL, SBP System NF1, SBP PL@NF1, SBP NF2, SBP PL@NF2, EM 122/124 0.5° 245 3.5 m 207 1749 m 180 EM 120/122/124 1° 242 3.5 m 210 438 m 189 EM 120/122/124 2° 236 3.5 m 210 110 m 195 EM 302 /304 MKI and MKII 0.5° 241 1.3 m 211 704 m 184 EM 300/302/304 MKI and MKII 1° 237 1.3 m 213 175 m 192 EM 300/302/304 MKI and MKII 2° 231 1.3 m 213 43 m 198 EM 710/712 0.5° 232 0.3 m 213 246 m 184 EM 710/712 1° 228 0.3 m 215 61 m 192 EM 710/712 2° 222 0.3 m 215 15 m 198 EM 3002 (1.5°) 216 0.01 m 227 8 m 198 EM 2040 (0.5°) 217 0.002 m 225 82 m 179 EM 2040 (1.0°) 211 0.002 m 225 20 m 185 EM 2040C (1.3°) 205 0.0005 m 227 11 m 183 Single Beam System SL TP BW TR RS 12Khz 12-16/60 18+1 Element 221,8 2000 16 171 -168,5 12Khz 12-16/60 1 Wide Element 197,8 100 60 142 -168,5 1

KeyboardInterrupt: 